In [232]:
import random
import numpy as np
random.seed(10)

def loss(y_true, y_pred):
    return np.sum((y_true - y_pred)**2)

def loss_d(y_true, y_pred):
    return 2 * y_pred - 2 * y_true

def sigm_fun(x):
    return 1 / (1 + np.exp(-x))
def sigm_d(sigm_val):
    return sigm_val * (1 - sigm_val)

class Neuron:
    def __init__(self, input_dim, activation_f = sigm_fun, activation_f_d = sigm_d):
        self.activation_f = activation_f
        self.activation_f_d = activation_f_d
        weight_list = []
        for i in range(input_dim):
            weight_list.append(random.uniform(-1, 1))
        self.weights = np.asarray(weight_list)
        self.b = random.uniform(-1, 1)
        self.b_grad = 0
        self.w_grad = np.zeros(input_dim)
        self.out_grad = 0
    
    def work(self, inputs):
        self.x = np.array(inputs)
        sum = np.sum(inputs * self.weights) + self.b
        self.output = self.activation_f(sum)
        return self.output

    def x_grad(self, i):
        return self.out_grad * self.activation_f_d(self.output) * self.weights[i]

    def parameters_grad(self):
        self.w_grad = self.out_grad * self.activation_f_d(self.output) * self.x
        self.b_grad = self.out_grad * self.activation_f_d(self.output)
            

class NeuralNet:
    def __init__(self, number_of_neurons_in_layer, number_of_layers, input_dim):
        self.input_layer = []
        for i in range(number_of_neurons_in_layer):
            self.input_layer.append(Neuron(input_dim))
        self.net = []
        for i in range(number_of_layers - 1):
            current_layer =[]
            for j in range(number_of_neurons_in_layer):
                current_layer.append(Neuron(number_of_neurons_in_layer))
            self.net.append(current_layer)
        self.output_layer = [Neuron(number_of_neurons_in_layer, activation_f=lambda x:x, activation_f_d=lambda x:1)]
                                     
    def predict(self, inputs):
        temp_inputs = []
        for neuron in self.input_layer:
            temp_inputs.append(neuron.work(inputs))
        temp_inputs2 = []
        for layer in self.net:
            for neuron in layer:
                temp_inputs2.append(neuron.work(temp_inputs))
            temp_inputs = temp_inputs2
            temp_inputs2 = []
        return self.output_layer[0].work(temp_inputs)
    
    def train(self, input, expected_value, learning_rate=0.05):
        for k in range(50):
            neurons = []
            y_pred = self.predict(input)
            error = loss(y_pred, expected_value)
            print(f"pred {y_pred}, error {error}")
            error_d = loss_d(y_pred=y_pred, y_true=expected_value)
            # output layer gradient
            self.output_layer[0].out_grad = error_d
            self.output_layer[0].parameters_grad()
            neurons.append(self.output_layer[0])
            next_layer = self.output_layer
            layers = self.net.copy()
            layers.append(self.input_layer)
            for layer in reversed(layers):
                for i, neuron in enumerate(layer):
                    neurons.append(neuron)
                    neuron.w_grad = 0
                    neuron.out_grad = 0
                    neuron.b_grad = 0
                    for next_neuron in next_layer:
                        neuron.out_grad += next_neuron.x_grad(i)
                    neuron.parameters_grad()
            for neuron in neurons:
                neuron.weights += -neuron.w_grad * learning_rate
                neuron.b += -neuron.b_grad * learning_rate

In [233]:
random.seed(10)
net = NeuralNet(number_of_layers=3,number_of_neurons_in_layer=3,input_dim=3)


net.train([2,3,5], 100)

pred 0.3729323143657661, error 9925.552615637944
pred 15.568238179422542, error 7128.722404126722
pred 37.30668688531872, error 3930.451509295468
pred 56.11150315596211, error 1926.2001552291238
pred 69.28771877371527, error 943.2442181224015
pred 78.51568166903078, error 461.5759341464199
pred 84.98219074102602, error 225.5345949389247
pred 89.51919161200425, error 109.84734446588209
pred 92.71069472512556, error 53.13397139031229
pred 94.96738583050492, error 25.327205379002617
pred 96.5776153070745, error 11.7127169863708
pred 97.73974464480656, error 5.108754270680607
pred 98.58024307560763, error 2.0157097243600925
pred 99.16926674587249, error 0.69011773951328
pred 99.55026014287031, error 0.20226593909102997
pred 99.77170184414013, error 0.05212004796901973
pred 99.8888723575044, error 0.01234935292663058
pred 99.94715041117573, error 0.002793079038894726
pred 99.97516032889219, error 0.0006170092607443166
pred 99.98839156533509, error 0.00013475575536942526
pred 99.994589615879